#Environment for 3D CartPole 🐶 🛺 🚀 🦾
In this notebook there's the environment for a 3 dimensional CartPole, as described in details in this other nothebook [here](https://colab.research.google.com/drive/1rERvVgBZAJpiBx0FEkAEzIYrDqreGG33?hl=it#scrollTo=b91MlSWYly5s) 🐧 📚

🔖 🦤 *Code based on the standard pybullet [CartPole environment](https://github.com/bulletphysics/bullet3/blob/master/examples/pybullet/gym/pybullet_envs/bullet/cartpole_bullet.py)*


---

---

* 🎮  **Actions space**: Continous action space, an array of two element indicating the force/acceleration to be applied on x,y (sign and abs value).

* 🎮 **Observations space**: 4 arrays, cart and pole positions and cart and pole velocities. Every array has 2 element,that are the directions of the plane or the interesting Euler angles. Let's make without any nested array, in order to avoid complainments.

  1. 🛑  **FORMAT** [cart position: x,y; Euler angles for pole: θ,ϕ; cart velocity: ẋ,ẏ; pole velocity: θ̇,ϕ̇] AKA
  
                    `[x,y, ẋ,ẏ, θ,ϕ, θ̇,ϕ̇̇]`
  I'm ignoring z coordinate and the tilt angle Ψ, cause the cart must stay on the plane, not move in vertical, nore tilt with respect to the plane.

  2. 🛑 **LIMITATIONS**: The cart can move whitin a plane, so only x and y are limited to the dimensions of the plane itself. The cart has so stay on the plane, so z=0.
   For the angles, the pole should not tilt with respect to the plane: it has to stay on the plane, that's why ψ = 0 always. The other angle limits are set to the ones of the classic cartpole problem.

            *   ϕ, θ whitin 12 gradi;
            *   x, y within 2.4 and 0;




In [ ]:
import os, inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(os.path.dirname(currentdir))
os.sys.path.insert(0, parentdir)

import logging
import math
import gym
from gym import spaces
from gym.utils import seeding
import numpy as np
import time
import subprocess
import pybullet as p2
import pybullet_data
from pybullet_utils import bullet_client as bc
from pkg_resources import parse_version

logger = logging.getLogger(__name__)


class RealMegaFufiEnv(gym.Env):
  metadata = {'render.modes': ['human', 'rgb_array'], 'video.frames_per_second': 50}

  def __init__(self, renders=False):
    # set the render
    self._renders = renders
    self._render_height = 200
    self._render_width = 320
    self._physics_client_id = -1

    #set limits for actions and observations
    self.threshold_radians = 12 * 2 * math.pi / 360
    self.threshold_plane = 2.4

    high = np.array([
        self.threshold_plane * 2, self.threshold_plane * 2,
        np.finfo(np.float32).max, np.finfo(np.float32).max,
        self.threshold_radians * 2, self.threshold_radians * 2,
        np.finfo(np.float32).max, np.finfo(np.float32).max,
    ])

    self.observation_space = spaces.Box(-high, high, dtype=np.float32)

    action_lim = np.finfo(np.float32).max
    # force may be applied in the 2 direction of the xy plane
    self.action_space = spaces.Box(low=-action_lim, high=action_lim, shape=(2,), dtype=np.float32)

    self.seed()
    # start the bullet physics server and initialize things up
    self.reset()
    self.viewer = None
    self._configure()

  def _configure(self, display=None):
    self.display = display

  def seed(self, seed=None):
    self.np_random, seed = seeding.np_random(seed)
    return [seed]

  def reset(self):
    #    print("-----------reset simulation---------------")
    if self._physics_client_id < 0:
      if self._renders:
        self._p = bc.BulletClient(connection_mode=p2.GUI)
      else:
        self._p = bc.BulletClient()
      self._physics_client_id = self._p._client

      p = self._p
      p.resetSimulation()


      # Define cartpole parameters
      mass_cart = 1
      cart_size = [0.5, 0.5, 0.2]
      mass_pole = 0.1
      pole_size = [0.05, 0.05, 1.0]
      cart_position = [0, 0, 0.1]  # Slightly above the ground
      pole_position = [0, 0, 0.5]  # on the center of the cart

      # Create the cart (base)
      cart_collision_shape = p.createCollisionShape(p.GEOM_BOX, halfExtents=[s / 2 for s in cart_size])
      cart_visual_shape = p.createVisualShape(p.GEOM_BOX, halfExtents=[s / 2 for s in cart_size], rgbaColor=[0, 1, 1, 1])

      # Create the pole (link)
      pole_collision_shape = p.createCollisionShape(p.GEOM_BOX, halfExtents=[s / 2 for s in pole_size])
      print(pole_collision_shape)
      pole_visual_shape = p.createVisualShape(p.GEOM_BOX, halfExtents=[s / 2 for s in pole_size], rgbaColor=[1, 0.5, 0, 1])
      print(pole_visual_shape)

      # Center of mass for the pole will be halfway along its length
      pole_inertial_frame_position = [0, 0, -pole_size[2] / 2]  # Center of mass is halfway along the pole


      # Create the cart-pole system with a spherical joint for full rotation
      self.cartpole_id = p.createMultiBody(
          baseMass=mass_cart,
          baseCollisionShapeIndex=cart_collision_shape,
          baseVisualShapeIndex=cart_visual_shape,
          basePosition=cart_position,
          baseInertialFramePosition=[0, 0, 0],  # Base Inertia (no offset needed for the cart)

          # Link 1: Pole with spherical joint
          linkMasses=[mass_pole],  # Pole's mass
          linkCollisionShapeIndices=[pole_collision_shape],  # Collision shape of the pole
          linkVisualShapeIndices=[pole_visual_shape],  # Visual shape of the pole
          linkPositions=[[0, 0, cart_size[2] / 2]],  # Pole position relative to the cart
          linkOrientations=[[0, 0, 0, 1]],  # Orientation of the pole
          linkInertialFramePositions=[pole_inertial_frame_position],  # Inertia offset for the pole
          linkInertialFrameOrientations=[[0, 0, 0, 1]],  # Inertial orientation of the pole
          linkParentIndices=[0],  # Linked to the cart (base index = 0)
          linkJointTypes=[p.JOINT_SPHERICAL],  # Spherical joint for full 3D rotation
          linkJointAxis=[[0, 0, 0]]  # Spherical joint doesn't need an axis definition
      )

      p.changeDynamics(self.cartpole, -1, linearDamping=0, angularDamping=0) # Base (cart)
      p.changeDynamics(self.cartpole, 0, linearDamping=0, angularDamping=0) # Link with index 0 (pole+spherical)

      self.timeStep = 0.02
      p.setJointMotorControl2(self.cartpole, 1, p.VELOCITY_CONTROL, force=0)
      p.setJointMotorControl2(self.cartpole, 0, p.VELOCITY_CONTROL, force=0)
      p.setGravity(0, 0, -9.8)
      p.setTimeStep(self.timeStep)
      p.setRealTimeSimulation(0)

    p = self._p
    randstate = self.np_random.uniform(low=-0.05, high=0.05, size=(1,))
    #giving him in order: cartpole z position (0) and its velocity
    p.resetJointState(self.cartpole, 0, 0, randstate[0])

    # Get the position and orientation of the cart (base)
    cart_position, cart_orientation = p.getBasePositionAndOrientation(cartpole_id)

    # Get the velocity (linear and angular) of the cart (base)
    cart_linear_velocity, cart_angular_velocity = p.getBaseVelocity(cartpole_id)

    # Get the position, orientation, and velocity (linear and angular) of the pole (link 1)
    pole_link_state = p.getLinkState(cartpole_id, 0, computeLinkVelocity=True)
    pole_position = pole_link_state[0]  # Position of the pole
    pole_orientation = pole_link_state[1]  # Orientation of the pole
    pole_linear_velocity = pole_link_state[6]  # Linear velocity of the pole
    pole_angular_velocity = pole_link_state[7]  # Angular velocity of the pole

    self.state = cart_position[0:2] + cart_linear_velocity[0:2] + pole_position[0:2] + pole_linear_velocity[0:2]
    return np.array(self.state)


  def step(self, action):
    p = self._p
    if self._discrete_actions:
      force = self.force_mag if action == 1 else -self.force_mag
    else:
      force = action[0]

    p.setJointMotorControl2(self.cartpole, 0, p.TORQUE_CONTROL, force=force)
    p.stepSimulation()

    self.state = p.getJointState(self.cartpole, 1)[0:2] + p.getJointState(self.cartpole, 0)[0:2]
    theta, theta_dot, x, x_dot = self.state

    done =  x < -self.x_threshold \
                or x > self.x_threshold \
                or theta < -self.theta_threshold_radians \
                or theta > self.theta_threshold_radians
    done = bool(done)
    reward = 1.0
    #print("state=",self.state)
    return np.array(self.state), reward, done, {}

  def render(self, mode='human', close=False):
    if mode == "human":
      self._renders = True
    if mode != "rgb_array":
      return np.array([])
    base_pos=[0,0,0]
    self._cam_dist = 2
    self._cam_pitch = 0.3
    self._cam_yaw = 0
    if (self._physics_client_id>=0):
      view_matrix = self._p.computeViewMatrixFromYawPitchRoll(
        cameraTargetPosition=base_pos,
        distance=self._cam_dist,
        yaw=self._cam_yaw,
        pitch=self._cam_pitch,
        roll=0,
        upAxisIndex=2)
      proj_matrix = self._p.computeProjectionMatrixFOV(fov=60,
             aspect=float(self._render_width) /
             self._render_height,
             nearVal=0.1,
             farVal=100.0)
      (_, _, px, _, _) = self._p.getCameraImage(
          width=self._render_width,
          height=self._render_height,
          renderer=self._p.ER_BULLET_HARDWARE_OPENGL,
          viewMatrix=view_matrix,
          projectionMatrix=proj_matrix)
    else:
      px = np.array([[[255,255,255,255]]*self._render_width]*self._render_height, dtype=np.uint8)
    rgb_array = np.array(px, dtype=np.uint8)
    rgb_array = np.reshape(np.array(px), (self._render_height, self._render_width, -1))
    rgb_array = rgb_array[:, :, :3]
    return rgb_array

  def configure(self, args):
    pass

  def close(self):
    if self._physics_client_id >= 0:
      self._p.disconnect()
    self._physics_client_id = -1